In [ ]:
!pip uninstall tensorflow

In [12]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [7]:
# -*- coding: utf-8 -*-
"""Sentiment analysis using CNN on financial news"""

# Install required libraries
# !pip install contractions pandas nltk bs4 scikit-learn setuptools tensorflow 

# Import necessary libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import re
import nltk
import tqdm
import unicodedata
import contractions
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
!pip install nltk

In [9]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jerry\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jerry\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
# Load and preprocess data
df = pd.read_csv('../data.csv', delimiter=',', encoding='latin-1', header=None, skiprows=1)
df = df.rename(columns=lambda x: ['Sentiment', 'Sentence'][x])
df = df[['Sentence', 'Sentiment']]
df = df[df.Sentiment != "neutral"]


In [11]:
df.head()

,Sentence,Sentiment
0,positive,The GeoSolutions technology will leverage Bene...
1,negative,"$ESI on lows, down $1.50 to $2.50 BK a real po..."
2,positive,"For the last quarter of 2010 , Componenta 's n..."
3,neutral,According to the Finnish-Russian Chamber of Co...
4,neutral,The Swedish buyout firm has sold its remaining...


In [22]:

!pip install newspaper3k lxml_html_clean
import newspaper
# Create a newspaper article object
article = newspaper.Article('https://www.chinadaily.com.cn/a/202403/29/WS66068ae1a31082fc043bf7a9.html')

# Download the article
article.download()

# Parse the article
article.parse()

# Get the article text
text = article.text

print(text)

  Using cached lxml_html_clean-0.1.1-py3-none-any.whl.metadata (1.5 kB)
Using cached lxml_html_clean-0.1.1-py3-none-any.whl (11 kB)
A panel discussion themed on "Digital Economy in Integration and Innovation" is held during the Boao Forum for Asia (BFA) Annual Conference 2024 in Boao, South China's Hainan province, March 28, 2024. [Photo/Xinhua]

BOAO, Hainan - While highlighting the remarkable achievements unlocked by the Belt and Road Initiative (BRI), participants at the Boao Forum for Asia (BFA) Annual Conference 2024 also called on participating countries to strengthen cooperation in technological innovation and the digital economy.

Since its inception in 2013, the China-proposed BRI has reaped substantial benefits from deepening the "hard connectivity" of infrastructure and the "soft connectivity" of technology, to coordination of standards and rules in its participating countries, said attendees at a panel discussion on high-quality Belt and Road cooperation on Friday.

The BRI

In [12]:

import newspaper
# Create a newspaper article object
article = newspaper.Article('https://www.chinadaily.com.cn/a/202403/29/WS66068ae1a31082fc043bf7a9.html')

# Download the article
article.download()

# Parse the article
article.parse()

# Get the article text
text = article.text

# Save the article text to a file
with open('financial_news.csv', 'w') as f:
  f.write(text)
# Data cleaning
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def stopwords_removal(words):
    list_stopwords = nltk.corpus.stopwords.words('english')
    return [word for word in words if word not in list_stopwords]

def pre_process_corpus(docs):
    norm_docs = []
    for doc in tqdm.tqdm(docs):
        # Case folding
        doc = doc.lower()
        # Remove special characters and whitespaces
        doc = strip_html_tags(doc)
        doc = doc.translate(doc.maketrans("\n\t\r", "   "))
        doc = remove_accented_chars(doc)
        doc = contractions.fix(doc)
        doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
        doc = re.sub(' +', ' ', doc)
        doc = doc.strip()
        # Tokenize
        doc = word_tokenize(doc)
        # Filtering
        doc = stopwords_removal(doc)
        norm_docs.append(doc)

    norm_docs = [" ".join(word) for word in norm_docs]
    return norm_docs

df.Sentence = pre_process_corpus(df.Sentence)


100%|██████████| 5842/5842 [00:04<00:00, 1200.44it/s]


In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE

# Load your dataset
# Assuming df contains your dataset

# One-hot encode the target variable
encoder = OneHotEncoder()
y_encoded = encoder.fit_transform(df_balance[['Sentence']])
y_encoded_df = pd.DataFrame(y_encoded.toarray(), columns=encoder.get_feature_names_out(['Sentence']))

# Prefix column names to avoid duplicates
y_encoded_df.columns = [f'sentiment_{col}' for col in y_encoded_df.columns]

# Concatenate df_balance and y_encoded_df
df_balance = pd.concat([df_balance, y_encoded_df], axis=1)

# Drop the original 'Sentence' column
df_balance.drop('Sentence', axis=1, inplace=True)

# Handle imbalanced data (oversampling)
smote = SMOTE(random_state=42)
X = df_balance.drop(y_encoded_df.columns, axis=1)
y = df_balance[y_encoded_df.columns]
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split data into train and test sets after resampling
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.1, random_state=42)

# Tokenize and pad sequences
token = Tokenizer()
token.fit_on_texts(X_train)
vocab = len(token.index_word) + 1

X_train = token.texts_to_sequences(X_train)
X_test = token.texts_to_sequences(X_test)

MAX_SEQUENCE_LENGTH = 30
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH, padding="post")
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH, padding="post")

# Encode labels
le = LabelEncoder()
num_classes = 2
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Build CNN model
vec_size = 300
model = Sequential()
model.add(Embedding(vocab, vec_size))
model.add(Conv1D(64, 8, activation="relu"))
model.add(MaxPooling1D(2))
model.add(Dropout(0.1))
model.add(Dense(8, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(4, activation='relu'))
model.add(Dropout(0.1))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=tf.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

# Train model
epochs = 100
batch_size = 4

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('./best_model/sentiment_analysis.keras', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
history = model.fit(X_train, y_train, batch_size=batch_size, shuffle=True, validation_split=0.1, epochs=epochs, verbose=1, callbacks=[es, mc])

# Evaluate model
train_acc = model.evaluate(X_train, y_train, verbose=1)
test_acc = model.evaluate(X_test, y_test, verbose=1)
print('Train: %.2f%%, Test: %.2f%%' % (train_acc[1]*100, test_acc[1]*100))

# Identify overfitting
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Confusion matrix
def predictions(x):
    prediction_probs = model.predict(x)
    predictions = [1 if prob > 0.5 else 0 for prob in prediction_probs]
    return predictions

labels = ['positive', 'negative']
print("CNN 1D Accuracy: %.2f%%" % (accuracy_score(y_test, predictions(X_test))*100))
print("CNN 1D Precision: %.2f%%" % (precision_score(y_test, predictions(X_test), average="macro")*100))
print("CNN 1D Recall: %.2f%%" % (recall_score(y_test, predictions(X_test), average="macro")*100))
print("CNN 1D f1_score: %.2f%%" % (f1_score(y_test, predictions(X_test), average="macro")*100))
print('================================================\n')
print(classification_report(y_test, predictions(X_test)))
pd.DataFrame(confusion_matrix(y_test, predictions(X_test)), index=labels, columns=labels)

# ROC AUC
def plot_roc_curve(y_test, y_pred):
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

plot_roc_curve(y_test, predictions(X_test))
print("Model AUC score: %.2f%%" % (roc_auc_score(y_test, predictions(X_test))*100))

# Save the trained model
model.save('sentiment_analysis_model.h5')

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [27]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define sentiment thresholds (modify these values if needed)
POSITIVE_THRESHOLD = 0.5
NEGATIVE_THRESHOLD = 1 - POSITIVE_THRESHOLD  # Threshold for negative sentiment

# Load the saved model
model = load_model('sentiment_analysis_model.h5')

# Define sample text
sample_text = []
with open('financial_news.csv', 'r') as file:
    for line in file:
        sample_text.append(line)

# Create tokenizer and fit on sample text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sample_text)

# Define maximum sequence length based on your model's configuration (replace with actual value)
MAX_SEQUENCE_LENGTH = 100  # Replace with appropriate value

# Preprocess sample text
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = pad_sequences(sample_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post")

# Predict sentiment
predictions = model.predict(sample_padded)

# Print results
for i, prediction in enumerate(predictions):
    if prediction[0] > POSITIVE_THRESHOLD:
        print(f"Sample {i+1}: Positive")
    elif prediction[0] < NEGATIVE_THRESHOLD:
        print(f"Sample {i+1}: Negative")
    else:
        print(f"Sample {i+1}: Neutral")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
Sample 1: Negative
Sample 2: Positive
Sample 3: Positive
Sample 4: Positive
Sample 5: Negative
Sample 6: Positive
Sample 7: Negative
Sample 8: Positive
Sample 9: Negative
Sample 10: Positive
Sample 11: Negative
Sample 12: Positive
Sample 13: Negative
Sample 14: Positive
Sample 15: Negative
Sample 16: Positive
Sample 17: Negative
Sample 18: Positive
Sample 19: Negative
Sample 20: Positive
Sample 21: Negative
Sample 22: Positive
Sample 23: Negative
Sample 24: Positive
Sample 25: Negative
